In [235]:
import requests
from datetime import datetime 

Api_key = "bbuUFrmvBachgADcHjZRxT8Bq2G4j46E8dFbWNgI"

asteroids_data = []            # intializing the empty list
target = 10000                    # required target
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-01-01&end_date=2025-01-07&api_key={Api_key}"
tab1 = []                     #table 1 list to push the data on asteriod table
tab2 = []                    #table 2 list to push the data on cloase_approach table
while len(asteroids_data) < target: # while loop is used to make calling api until target reached
    response = requests.get(url)   # get() used tp call url and stored in variable
    data = response.json()         #json format data as response
    details = data['near_earth_objects']      # here we takes the NEO data from json
    for date,info in details.items():         # item() method is used to return a view object and contain key value pair of dict as tuples
      for i in info:                          # we loop thro every asteriod in the list
        asteroids_data.append(dict(id = i['id'], #neo_reference_id same as id
        name = i['name'],
        mag= float(i['absolute_magnitude_h']),
        dia_min = float(i['estimated_diameter']['kilometers']['estimated_diameter_min']),
        dia_max = float(i['estimated_diameter']['kilometers']['estimated_diameter_max']),
        hazard = bool(i['is_potentially_hazardous_asteroid']),
        date = datetime.strptime(i['close_approach_data'][0]['close_approach_date'],"%Y-%m-%d"),
        velocity = float(i['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
        astronomical = float(i['close_approach_data'][0]['miss_distance']['astronomical']),
        missed_dist_kilo = float(i['close_approach_data'][0]['miss_distance']['kilometers']),                   
        lunar_dist =  float(i['close_approach_data'][0]['miss_distance']['lunar']),
        orbiting_body = (i['close_approach_data'][0]['orbiting_body'])
        ))

        tab1.append((              # append the data into table 1 - asteriod table
         i['id'], #neo_reference_id same as id
         i['name'],
         float(i['absolute_magnitude_h']),
         float(i['estimated_diameter']['kilometers']['estimated_diameter_min']),
         float(i['estimated_diameter']['kilometers']['estimated_diameter_max']),
         bool(i['is_potentially_hazardous_asteroid'])
         
        ))

        tab2.append((             # append the data into table 2 - close_approach table
         i['id'], #neo_reference_id same as id
         datetime.strptime(i['close_approach_data'][0]['close_approach_date'],"%Y-%m-%d"),
         float(i['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
         float(i['close_approach_data'][0]['miss_distance']['astronomical']),
         float(i['close_approach_data'][0]['miss_distance']['kilometers']),                   
         float(i['close_approach_data'][0]['miss_distance']['lunar']),
         (i['close_approach_data'][0]['orbiting_body'])
        ))
          
        if len(asteroids_data) >= target:    #condition will stop the loop if it reaches the target
            break
      if len(asteroids_data) >= target:     #two condition will break the loop if it reaches the target
          break

url = data['links'].get('next')            # pagination happens, next will collect the next page link from response.


In [237]:
len(asteroids_data)

10000

In [233]:
asteroids_data[0]

{'id': '2226514',
 'name': '226514 (2003 UX34)',
 'mag': 20.14,
 'dia_min': 0.2492039814,
 'dia_max': 0.5572370428,
 'hazard': True,
 'date': datetime.datetime(2025, 1, 7, 0, 0),
 'velocity': 59576.37781234,
 'astronomical': 0.1236081659,
 'missed_dist_kilo': 18491518.333246633,
 'lunar_dist': 48.0835765351,
 'orbiting_body': 'Earth'}

In [61]:
!pip install pymysql

In [124]:
import pymysql

In [126]:
connection = pymysql.connect(

                            host = "localhost", # IP address of your server
                            user = "rahila",
                            password = "12345",
                            database = "asteriod"
                           
                    )
cursor = connection.cursor()

In [128]:
connection

In [130]:
cursor.execute("""create table asteroids 
                (id INT, 
                name varchar(300), 
                absolute_magnitude_h FLOAT, 
                estimated_diameter_min_km FLOAT, 
                estimated_diameter_max_km FLOAT,
                is_potentially_hazardous_asteroid BOOL);""")

0

In [132]:
cursor.execute("""create table close_approach 
                (id INT, 
                close_approach_date DATE, 
                relative_velocity_kmph FLOAT, 
                astronomical FLOAT, 
                miss_distance_km FLOAT,
                miss_distance_lunar FLOAT,
                orbiting_body varchar(300));""")

0

In [134]:
insert = """
insert into asteroids
values
(%s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert,tab1)

connection.commit()

In [136]:
insert = """
insert into close_approach
values
(%s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert,tab2)

connection.commit()